# Initial machine
Objective of this is to build a simple method that runs A* and can find some shortest path between two nodes.

In [94]:
import sys
import os
#path = os.path.abspath("datasets")
sys.path.append('../')
import data_readers

import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from scipy.spatial import distance
import gensim.downloader
import networkx as nx
from transformers import BertTokenizer, BertModel

from sklearn.metrics.pairwise import cosine_similarity
import math

In [58]:
glove_vectors = gensim.downloader.load('glove-twitter-25')

# The links and edges
wikispeedia= nx.read_edgelist('../datasets/wikispeedia_paths-and-graph/links.tsv',
                                    create_using=nx.DiGraph)

In [82]:
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to get embeddings
def get_embedding(text: str):
    temp_str = text.replace('_', ' ')
    inputs = tokenizer(temp_str, return_tensors="pt")
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

C:\Users\lolco\anaconda3\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lolco\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [114]:
# For the heuristic, I need to create a function for two reasons:
# A) I need to parse the node and make sure I'm getting the title as the value
# B) I need to take care of cases where the word doesn't exist.
# Carlos already kinda took care of that!

def distance_two_words(w1: str, w2: str):
    """Receives a string that was in the wikispeedia dataset, and transforms it as needed to work
    with word2vec.

    Namly, it splits words """


    embedding1 = get_embedding(w1)
    embedding2 = get_embedding(w2)
    similarity = cosine_similarity(embedding1.detach().numpy(), embedding2.detach().numpy())[0][0]
    # Adding absolute, just in case it is needed
    # Similarity is actually 1 - abs(similarity) + 1,
    # As we want closer words to have a smaller distance
    # The last plus one is to indicate that there would be an extra cost to exploring...
    #   not sure if it's needed though...
    similarity = 1 -abs(similarity) + 1
    # print("First word:", w1, ". Second word:", w2, ". GoodDistance:", similarity)
    return similarity

In [119]:
nx.astar_path(wikispeedia, 'Actor', 'Japan', heuristic=distance_two_words)

First word: Charles_II_of_England . Second word: Japan . GoodDistance: 1.6090421080589294
First word: Drama . Second word: Japan . GoodDistance: 1.309839129447937
First word: Film . Second word: Japan . GoodDistance: 1.2666865587234497
First word: Greece . Second word: Japan . GoodDistance: 1.2139631509780884
First word: H%C3%A4nsel_und_Gretel . Second word: Japan . GoodDistance: 1.7219897210597992
First word: Japan . Second word: Japan . GoodDistance: 1.0
First word: Latin . Second word: Japan . GoodDistance: 1.27892005443573
First word: Middle_Ages . Second word: Japan . GoodDistance: 1.536699801683426
First word: Opera . Second word: Japan . GoodDistance: 1.2991307973861694
First word: Television . Second word: Japan . GoodDistance: 1.2993553280830383
First word: The_Marriage_of_Figaro . Second word: Japan . GoodDistance: 1.5767060220241547
First word: The_Simpsons . Second word: Japan . GoodDistance: 1.3803905844688416
First word: William_Shakespeare . Second word: Japan . GoodDist

['Actor', 'Japan']

Method works, but holy shit it's slow af...

The shortest path found tends to be good enough, but it's actually not!

This method is finding a path that is shortest than the one described in the shortest path dictionary... how?

In the printed example, Actor and Japan are neighbors, but in the dataset it says their distance is of 3!

I'll double check the data to see if it's all correct or not

In [121]:
wikispeedia.adj['Actor']

AtlasView({'Charles_II_of_England': {}, 'Drama': {}, 'Film': {}, 'Greece': {}, 'H%C3%A4nsel_und_Gretel': {}, 'Japan': {}, 'Latin': {}, 'Middle_Ages': {}, 'Opera': {}, 'Television': {}, 'The_Marriage_of_Figaro': {}, 'The_Simpsons': {}, 'William_Shakespeare': {}})

And japan is a neighbor... great!

Proof that this shit is wrong follows, reading in the shortest path df and parsing it as indicated

And it's read as well as possible... fuckkkk

In [122]:
shortest_path = np.genfromtxt("../datasets/wikispeedia_paths-and-graph/shortest-path-distance-matrix.txt",
                                  delimiter=1, missing_values=-1, dtype=int)
articles = pd.read_csv('../datasets/wikispeedia_paths-and-graph/articles.tsv', sep='\t', skiprows=12,
                       names=["article_name"])

shortest_path_df = pd.DataFrame(shortest_path, index=articles.values, columns=articles.values)

In [132]:
shortest_path_df[('Actor',)][('Japan',)]

3